In [7]:
from __future__ import print_function
import argparse
import os
import torch
import torch.utils.data
from torch import nn, optim
from torch.autograd import Variable
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
from dotmap import DotMap

import torch.nn.functional as F


class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

args=DotMap()
args.batch_size=128
args.saveInt=1
args.epochs=20
args.cuda=True
args.seed=1
args.log_interval=10
args.outf='./vae_mnist'

args.cuda = not args.no_cuda and torch.cuda.is_available()
try:
    os.makedirs(args.outf)
except OSError:
    pass


torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)



kwargs = {'num_workers': 1, 'pin_memory': True} if args.cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=args.batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
    batch_size=args.batch_size, shuffle=True, **kwargs)


class VAE_E(nn.Module):
    def __init__(self):
        super(VAE_E, self).__init__()

        self.fc1 = nn.Linear(784, 400)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)

        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def encode(self, x):
        h1 = self.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        if self.training:
          std = logvar.mul(0.5).exp_()
          eps = Variable(std.data.new(std.size()).normal_())
          return eps.mul(std).add_(mu)
        else:
          return mu

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 784))
        z = self.reparameterize(mu, logvar)
        return z, mu, logvar

class VAE_D(nn.Module):
    def __init__(self):
        super(VAE_D, self).__init__()
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 784)

        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    
    def forward(self,z):
        h3 = self.relu(self.fc3(z))
        return self.sigmoid(self.fc4(h3))
    
    
class Net_Classifier(nn.Module):
    def __init__(self):
        super(Net_Classifier, self).__init__()
        self.fc1 = nn.Linear(20, 100)
        self.fc2 = nn.Linear(100, 10)

    def forward(self, x):
        x=F.relu(self.fc1(x))
        x=self.fc2(x)
        return F.log_softmax(x)

model_E = VAE_E()
model_D = VAE_D()
net_classifier=Net_Classifier()    

if args.cuda:
    model_E.cuda()
    model_D.cuda()
    net_classifier.cuda()


def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784))

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    # Normalise by same number of elements as in reconstruction
    KLD /= args.batch_size * 784

    return BCE + KLD


optimizer_E = optim.Adam(model_E.parameters(), lr=1e-3)
optimizer_D = optim.Adam(model_D.parameters(), lr=1e-3)
optimizer_classifier = optim.Adam(net_classifier.parameters(), lr=1e-3)     

def train(epoch):
    model_E.train()
    model_D.train()
    net_classifier.train()
    train_loss = 0
    train_loss_classifier=0
    for batch_idx, (data, target) in enumerate(train_loader):
        if args.cuda:
            data = data.cuda()
            target=target.cuda()
        data, target = Variable(data), Variable(target)    
        optimizer_E.zero_grad()
        optimizer_D.zero_grad()
        optimizer_classifier.zero_grad()
        z, mu, logvar = model_E(data)
        recon_batch=model_D(z)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward(retain_graph=True)
        train_loss += loss.data[0]
        
        output = net_classifier(z)
        train_loss_classifier = F.nll_loss(output, target)
        train_loss_classifier.backward() 
        
        optimizer_E.step()
        optimizer_D.step()
        optimizer_classifier.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.data[0] / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))
    print('====> Epoch: {} Average Classifier loss: {:.4f}'.format(
          epoch, train_loss_classifier.data[0] / len(train_loader.dataset)))

def test(epoch):
    model_E.eval()
    model_D.eval()
    net_classifier.eval()
    test_loss = 0
    test_loss_classifier=0
    correct=0
    for i, (data, target) in enumerate(test_loader):
        if args.cuda:
            data = data.cuda()
            target=target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)        
        z, mu, logvar = model_E(data)
        recon_batch = model_D(z)
        test_loss += loss_function(recon_batch, data, mu, logvar).data[0]
        
        output = net_classifier(z)
        test_loss_classifier += F.nll_loss(output, target, size_average=False).data[0] # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        
        if i == 0:
          n = min(data.size(0), 8)
          comparison = torch.cat([data[:n],
                                  recon_batch.view(args.batch_size, 1, 28, 28)[:n]])
          save_image(comparison.data.cpu(),
                     args.outf+'/reconstruction_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))
    test_loss_classifier /= len(test_loader.dataset)
    print('\nTest set classifier: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss_classifier, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


for epoch in range(1, args.epochs + 1):
    train(epoch)
    test(epoch)
    sample = Variable(torch.randn(64, 20))
    if args.cuda:
       sample = sample.cuda()
    sample = model_D(sample).cpu()
    save_image(sample.data.view(64, 1, 28, 28),
               args.outf+'/sample_' + str(epoch) + '.png')

    if epoch%args.saveInt == 0 and epoch!=0:
        torch.save(model_E.state_dict(), '%s/vae_e_epoch_%d.pth' % (args.outf, epoch))
    if epoch%args.saveInt == 0 and epoch!=0:
        torch.save(model_D.state_dict(), '%s/vae_d_epoch_%d.pth' % (args.outf, epoch))    

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.005476
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.003236
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.002412
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.002489
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.002895
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.002896
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.002650
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.002679
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.002488
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.002421
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.002412
Train Epoch: 1 [14080/60000 (23%)]	Loss: 0.002351
Train Epoch: 1 [15360/60000 (26%)]	Loss: 0.002234
Train Epoch: 1 [16640/60000 (28%)]	Loss: 0.002148
Train Epoch: 1 [17920/60000 (30%)]	Loss: 0.002078
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.002147
Train Epoch: 1 [20480/60000 (34%)]	Loss: 0.002147
Train Epoch: 1 [21760/60000 (36%)]	Loss: 0.002117
Train Epoch: 1 [23040/60000 (38%)]	Loss: 0.002105
Train Epoch: 1 [24320/60000 (41%)]	Loss: 0.002089
Train Epoch: 1 [

Train Epoch: 4 [19200/60000 (32%)]	Loss: 0.001394
Train Epoch: 4 [20480/60000 (34%)]	Loss: 0.001395
Train Epoch: 4 [21760/60000 (36%)]	Loss: 0.001384
Train Epoch: 4 [23040/60000 (38%)]	Loss: 0.001398
Train Epoch: 4 [24320/60000 (41%)]	Loss: 0.001403
Train Epoch: 4 [25600/60000 (43%)]	Loss: 0.001368
Train Epoch: 4 [26880/60000 (45%)]	Loss: 0.001395
Train Epoch: 4 [28160/60000 (47%)]	Loss: 0.001384
Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.001389
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.001416
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.001340
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.001385
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.001352
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.001377
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.001410
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.001386
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.001388
Train Epoch: 4 [40960/60000 (68%)]	Loss: 0.001370
Train Epoch: 4 [42240/60000 (70%)]	Loss: 0.001410
Train Epoch: 4 [43520/60000 (72%)]	Loss: 0.001314


Train Epoch: 7 [35840/60000 (60%)]	Loss: 0.001294
Train Epoch: 7 [37120/60000 (62%)]	Loss: 0.001291
Train Epoch: 7 [38400/60000 (64%)]	Loss: 0.001298
Train Epoch: 7 [39680/60000 (66%)]	Loss: 0.001372
Train Epoch: 7 [40960/60000 (68%)]	Loss: 0.001282
Train Epoch: 7 [42240/60000 (70%)]	Loss: 0.001308
Train Epoch: 7 [43520/60000 (72%)]	Loss: 0.001294
Train Epoch: 7 [44800/60000 (75%)]	Loss: 0.001288
Train Epoch: 7 [46080/60000 (77%)]	Loss: 0.001288
Train Epoch: 7 [47360/60000 (79%)]	Loss: 0.001273
Train Epoch: 7 [48640/60000 (81%)]	Loss: 0.001294
Train Epoch: 7 [49920/60000 (83%)]	Loss: 0.001326
Train Epoch: 7 [51200/60000 (85%)]	Loss: 0.001327
Train Epoch: 7 [52480/60000 (87%)]	Loss: 0.001290
Train Epoch: 7 [53760/60000 (90%)]	Loss: 0.001346
Train Epoch: 7 [55040/60000 (92%)]	Loss: 0.001267
Train Epoch: 7 [56320/60000 (94%)]	Loss: 0.001273
Train Epoch: 7 [57600/60000 (96%)]	Loss: 0.001279
Train Epoch: 7 [58880/60000 (98%)]	Loss: 0.001289
====> Epoch: 7 Average loss: 0.0013
====> Epoch: 7

Train Epoch: 10 [52480/60000 (87%)]	Loss: 0.001256
Train Epoch: 10 [53760/60000 (90%)]	Loss: 0.001266
Train Epoch: 10 [55040/60000 (92%)]	Loss: 0.001256
Train Epoch: 10 [56320/60000 (94%)]	Loss: 0.001236
Train Epoch: 10 [57600/60000 (96%)]	Loss: 0.001256
Train Epoch: 10 [58880/60000 (98%)]	Loss: 0.001283
====> Epoch: 10 Average loss: 0.0013
====> Epoch: 10 Average Classifier loss: 0.0000
====> Test set loss: 0.0012

Test set classifier: Average loss: 0.0780, Accuracy: 9782/10000 (98%)

Train Epoch: 11 [0/60000 (0%)]	Loss: 0.001261
Train Epoch: 11 [1280/60000 (2%)]	Loss: 0.001249
Train Epoch: 11 [2560/60000 (4%)]	Loss: 0.001290
Train Epoch: 11 [3840/60000 (6%)]	Loss: 0.001226
Train Epoch: 11 [5120/60000 (9%)]	Loss: 0.001186
Train Epoch: 11 [6400/60000 (11%)]	Loss: 0.001198
Train Epoch: 11 [7680/60000 (13%)]	Loss: 0.001283
Train Epoch: 11 [8960/60000 (15%)]	Loss: 0.001240
Train Epoch: 11 [10240/60000 (17%)]	Loss: 0.001246
Train Epoch: 11 [11520/60000 (19%)]	Loss: 0.001308
Train Epoch: 11

Train Epoch: 14 [1280/60000 (2%)]	Loss: 0.001189
Train Epoch: 14 [2560/60000 (4%)]	Loss: 0.001248
Train Epoch: 14 [3840/60000 (6%)]	Loss: 0.001213
Train Epoch: 14 [5120/60000 (9%)]	Loss: 0.001232
Train Epoch: 14 [6400/60000 (11%)]	Loss: 0.001226
Train Epoch: 14 [7680/60000 (13%)]	Loss: 0.001302
Train Epoch: 14 [8960/60000 (15%)]	Loss: 0.001247
Train Epoch: 14 [10240/60000 (17%)]	Loss: 0.001203
Train Epoch: 14 [11520/60000 (19%)]	Loss: 0.001271
Train Epoch: 14 [12800/60000 (21%)]	Loss: 0.001264
Train Epoch: 14 [14080/60000 (23%)]	Loss: 0.001220
Train Epoch: 14 [15360/60000 (26%)]	Loss: 0.001211
Train Epoch: 14 [16640/60000 (28%)]	Loss: 0.001240
Train Epoch: 14 [17920/60000 (30%)]	Loss: 0.001240
Train Epoch: 14 [19200/60000 (32%)]	Loss: 0.001205
Train Epoch: 14 [20480/60000 (34%)]	Loss: 0.001199
Train Epoch: 14 [21760/60000 (36%)]	Loss: 0.001199
Train Epoch: 14 [23040/60000 (38%)]	Loss: 0.001257
Train Epoch: 14 [24320/60000 (41%)]	Loss: 0.001227
Train Epoch: 14 [25600/60000 (43%)]	Loss: 

Train Epoch: 17 [15360/60000 (26%)]	Loss: 0.001158
Train Epoch: 17 [16640/60000 (28%)]	Loss: 0.001224
Train Epoch: 17 [17920/60000 (30%)]	Loss: 0.001177
Train Epoch: 17 [19200/60000 (32%)]	Loss: 0.001177
Train Epoch: 17 [20480/60000 (34%)]	Loss: 0.001233
Train Epoch: 17 [21760/60000 (36%)]	Loss: 0.001169
Train Epoch: 17 [23040/60000 (38%)]	Loss: 0.001245
Train Epoch: 17 [24320/60000 (41%)]	Loss: 0.001181
Train Epoch: 17 [25600/60000 (43%)]	Loss: 0.001177
Train Epoch: 17 [26880/60000 (45%)]	Loss: 0.001217
Train Epoch: 17 [28160/60000 (47%)]	Loss: 0.001228
Train Epoch: 17 [29440/60000 (49%)]	Loss: 0.001205
Train Epoch: 17 [30720/60000 (51%)]	Loss: 0.001214
Train Epoch: 17 [32000/60000 (53%)]	Loss: 0.001174
Train Epoch: 17 [33280/60000 (55%)]	Loss: 0.001207
Train Epoch: 17 [34560/60000 (58%)]	Loss: 0.001177
Train Epoch: 17 [35840/60000 (60%)]	Loss: 0.001203
Train Epoch: 17 [37120/60000 (62%)]	Loss: 0.001180
Train Epoch: 17 [38400/60000 (64%)]	Loss: 0.001214
Train Epoch: 17 [39680/60000 (6

Train Epoch: 20 [28160/60000 (47%)]	Loss: 0.001206
Train Epoch: 20 [29440/60000 (49%)]	Loss: 0.001190
Train Epoch: 20 [30720/60000 (51%)]	Loss: 0.001233
Train Epoch: 20 [32000/60000 (53%)]	Loss: 0.001241
Train Epoch: 20 [33280/60000 (55%)]	Loss: 0.001177
Train Epoch: 20 [34560/60000 (58%)]	Loss: 0.001255
Train Epoch: 20 [35840/60000 (60%)]	Loss: 0.001214
Train Epoch: 20 [37120/60000 (62%)]	Loss: 0.001200
Train Epoch: 20 [38400/60000 (64%)]	Loss: 0.001187
Train Epoch: 20 [39680/60000 (66%)]	Loss: 0.001244
Train Epoch: 20 [40960/60000 (68%)]	Loss: 0.001201
Train Epoch: 20 [42240/60000 (70%)]	Loss: 0.001186
Train Epoch: 20 [43520/60000 (72%)]	Loss: 0.001197
Train Epoch: 20 [44800/60000 (75%)]	Loss: 0.001219
Train Epoch: 20 [46080/60000 (77%)]	Loss: 0.001250
Train Epoch: 20 [47360/60000 (79%)]	Loss: 0.001233
Train Epoch: 20 [48640/60000 (81%)]	Loss: 0.001219
Train Epoch: 20 [49920/60000 (83%)]	Loss: 0.001194
Train Epoch: 20 [51200/60000 (85%)]	Loss: 0.001231
Train Epoch: 20 [52480/60000 (8

In [60]:
import random
import numpy as np
i=0
for data,target in test_loader:
    if args.cuda:
        data, target = data[0].unsqueeze(1).cuda(), target.unsqueeze(1)[0].cuda()
    data, target = Variable(data, volatile=True), Variable(target)
    z, mu, logvar = model_E(data)
    recon_batch=model_D(z)
    output = net_classifier(z)
    pred = output.data.max(1, keepdim=True)
    res=pred[1].cpu()[0][0]
    adv_targets=set(list(range(0,10)))-set([res])
    adv_target=random.sample(adv_targets, 1)[0]
    #print(output)
    
    
    # Generate Adversarial Image 
    x = Variable(data.data, requires_grad=True)
    y_true=target
    SoftmaxWithXent = nn.CrossEntropyLoss()
    # Forward pass
    z, mu , logvar=model_E(x)
    outputs = net_classifier(z)
    loss = SoftmaxWithXent(outputs, y_true)
    loss.backward() # obtain gradients on x

    # Add perturbation
    epsilon = 0.1 
    x_grad   = torch.sign(x.grad.data)
    x_adversarial = torch.clamp(x.data + epsilon * x_grad, 0, 1) 

    if args.cuda:
        x_adversarial=x_adversarial.cuda()
    x_adversarial=Variable(x_adversarial)    
    # Classification after optimization  
    z_adv, mu_adv , logvar_adv=model_E(x_adversarial)
    y_pred_adversarial = np.argmax(net_classifier(z).cpu().data.numpy())

    if res!=y_pred_adversarial:
        print(y_true)
        print(res)
        print(y_pred_adversarial)
    
        save_image(data.data.cpu(),'test_%d.png' % (i))
        save_image(recon_batch.view(1,28,28).data.cpu(),'recon_%d.png' % i)
        save_image(x_adversarial.view(1,28,28).data.cpu(),'adv_img_%d.png' % i)
        i+=1
